# A Nuvem Inteligente
Em uma empresa de tecnologia, a equipe técnica enfrentava um grande desafio: alocar eficientemente recursos de CPU e memória nos servidores da nuvem para atender às demandas dos clientes. A solução manual estava se tornando impraticável e cara.

# O Desafio
A equipe precisava encontrar uma maneira de minimizar os custos operacionais e garantir tempos de resposta rápidos, ajustando dinamicamente a alocação de recursos em um ambiente de nuvem.

# A Solução Genética
Eles decidiram usar um algoritmo genético para resolver o problema. Este algoritmo funciona imitando a evolução natural:

**Inicialização:** Começa com várias configurações aleatórias de recursos dos servidores.

**Avaliação:** Calcula o desempenho de cada configuração, considerando tempo de resposta e custo.

**Seleção:** Escolhe as melhores configurações.

**Cruzamento e Mutação:** Combina e modifica ligeiramente essas configurações para criar novas.

**Iteração:** Repete o processo várias vezes, melhorando a cada geração.

**O Resultado**
Após várias iterações, o algoritmo encontrou configurações ótimas que usavam os recursos de maneira eficiente, reduzindo custos e melhorando os tempos de resposta. A nuvem agora ajustava-se automaticamente às demandas dos clientes.

# Conclusão
Com o algoritmo genético, a equipe técnica conseguiu otimizar a alocação de recursos na nuvem, tornando o sistema mais eficiente e econômico.

In [2]:
import random
import math
import copy
from typing import List, Tuple

min_cpu_capacity = 1 #Min de CPU configurável
max_cpu_capacity = 100 #Max de CPU configurável
min_memory_capacity = 8 #Min de RAM configurável
max_memory_capacity = 1000 #Max de RAM configurável

fixed_cost = 50.0 #Custo Fixo
cpu_cost = 0.5 #Custo fixo de CPU
memory_cost = 0.2 #Custo fixo de RAM

#Gerar demandas de clientes de forma aleatórias
def generate_client_demands(num_clients: int) -> List[Tuple[int, int]]:
    client_demands = []
    for _ in range(num_clients):
        cpu_demand = random.randint(1, 90)  # Demanda de CPU (unidades)
        memory_demand = random.randint(1, 700)  # Demanda de memória (GB)
        client_demands.append((cpu_demand, memory_demand))
    return client_demands

#Gerar populacão de "servidores" de forma aleatória (cpu,ram)
def generate_initial_population(population_size: int, min_cpu_capacity: int, max_cpu_capacity: int, min_memory_capacity: int, max_memory_capacity: int) -> List[Tuple[int, int]]:
    population = []
    for _ in range(population_size):
        cpu_capacity = random.randint(min_cpu_capacity, max_cpu_capacity)
        memory_capacity = random.randint(min_memory_capacity, max_memory_capacity)
        population.append((cpu_capacity, memory_capacity))
    return population


#Calculo do tempo de resposta
def calculate_response_time(cpu_demand: int, memory_demand: int, server_capacity: Tuple[int, int]) -> float:
    #Calcula a porcentagem de utilizacão da CPU
    cpu_utilization = cpu_demand / server_capacity[0]
    #Calcula a porcentagem de utilizacão da RAM
    memory_utilization = memory_demand / server_capacity[1]
    #Coleta o maior valor entre as 2 utilizacões
    utilization = max(cpu_utilization, memory_utilization)
    base_time = 10  # Tempo base para processamento (ms)

    #Calcula o tempo de resposta baseado na utilizacão maior.
    response_time = base_time / (1 - utilization) if utilization < 1 else float('inf')

    return response_time

#Calculo do custo se baseando na configuracão do servidor passado e os custos fixos
def calculate_operational_cost(server_capacity: Tuple[int, int], fixed_cost: float, cpu_cost: float, memory_cost: float) -> float:
    return fixed_cost + (server_capacity[0] * cpu_cost) + (server_capacity[1] * memory_cost)

#Cálculo do fitness (A soma do tempo de resposta + custo operacional, sendo 70% para tempo de resposta e 30% para custo os pesos)
def calculate_fitness(server_capacity: Tuple[int, int], client_demands: List[Tuple[int, int]], fixed_cost: float, cpu_cost: float, memory_cost: float) -> float:
    total_response_time = 0
    #Loop para passar as demandas dos clientes que foram geradas
    for cpu_demand, memory_demand in client_demands:
        #Calculo do tempo de resposta do servidor atual baseado nas demandas dos clientes
        total_response_time += calculate_response_time(cpu_demand, memory_demand, server_capacity)
    #média final
    average_response_time = total_response_time / len(client_demands)
    #custo operacional baseado no servidor atual (individuo)
    operational_cost = calculate_operational_cost(server_capacity, fixed_cost, cpu_cost, memory_cost)

    # Peso de 70% para o tempo de resposta e 30% para o custo operacional
    weighted_fitness = 0.7 * average_response_time + 0.3 * operational_cost

    return weighted_fitness

#Cruzamento
def crossover(parent1: Tuple[int, int], parent2: Tuple[int, int]) -> Tuple[int, int]:
    #Sorteia aleatoriamente número de 0 a 1
    crossover_point = random.randint(0, 1)
    if crossover_point == 0:
        #Se caso for zero é invertido a posicão 0 pega do parent1 e a posicão 1 do parent2 (cpu,ram)
        return (parent1[0], parent2[1])
    else:
        #Ao contrário pega a posicão 0 do parent2 e a posicão 1 do parent1 (cpu,ram)
        return (parent2[0], parent1[1])

#Mutacão
def mutate(individual: Tuple[int, int], mutation_probability: float, min_cpu_capacity: int, max_cpu_capacity: int, min_memory_capacity: int, max_memory_capacity: int) -> Tuple[int, int]:
    r = random.random()
    #Se caso número aleatório for abaixo da taxa de mutacão
    if r < mutation_probability:
        #é gerado novamente de forma aleatória novas CPU e RAM para o servidor filho (child)
        new_cpu_capacity = random.randint(min_cpu_capacity, max_cpu_capacity)
        new_memory_capacity = random.randint(min_memory_capacity, max_memory_capacity)
        return (new_cpu_capacity, new_memory_capacity)
    else:
        #Ao contrário simplesmente retorna o individuo como ele realmente é e somente com o cruzamento feito.
        return individual


def sort_population(population: List[List[Tuple[float, float]]], fitness: List[float]) -> Tuple[List[List[Tuple[float, float]]], List[float]]:

    combined_lists = list(zip(population, fitness))

    #Organiza a populacão de individuos com base na ordem crescente do fitness
    sorted_combined_lists = sorted(combined_lists, key=lambda x: x[1])

    #Separa as listas novamente em population e fitness
    sorted_population, sorted_fitness = zip(*sorted_combined_lists)

    return sorted_population, sorted_fitness


if __name__ == '__main__':

    POPULATION_SIZE = 100
    N_GENERATIONS = 500
    MUTATION_PROBABILITY = 0.8

    # Cria a populacão inicial de servidores de forma aleatória
    population = generate_initial_population(population_size=POPULATION_SIZE,min_cpu_capacity=min_cpu_capacity,max_cpu_capacity=max_cpu_capacity,min_memory_capacity=min_memory_capacity,max_memory_capacity=max_memory_capacity)

    # Variaveis para guardar os melhores resultados
    melhor_fitness_values = []
    melhores_solucoes = []
    melhores_solucoes.append((0,0))

    # Gerar demandas dos clientes
    client_demands = generate_client_demands(100)
    print(f"Client Demands {client_demands}")
    print(f"Server Capacities {population}")

    #Para cada geracão
    for generation in range(N_GENERATIONS):

        #É feito o cálculo do fitness de cada individuo, passando por cada um dos servidores gerados aleatoriamente no inicio do algoritmo.
        population_fitness = [calculate_fitness(individual,client_demands, fixed_cost, cpu_cost, memory_cost) for individual in population]

        #Organiza em forma crescente baseado no fitness
        population, population_fitness = sort_population(population,  population_fitness)


        #Salva o melhor fitness e a melhor solucão, calcula o fitness baseado na posicão [0] da population que em ordem crescente seria a menor.
        melhor_fitness = calculate_fitness(population[0],client_demands, fixed_cost, cpu_cost, memory_cost)
        melhor_solucao = population[0]

        #Verificamos aqui se a melhor_solucao atual é diferente da última melhor solucão que salvamos em melhores_solucoes[]
        if melhor_solucao != melhores_solucoes[-1]:
          #Se sim, printamos que existe uma nova melhor solucão e qual é o seu fitness e com separadores para serem destacados no nosso console.
          print(f"\n//////\nNova Melhor: {melhor_solucao}")
          print(f"Geracão {generation}: Melhor fitness = {melhor_fitness} Melhor solucão = {melhor_solucao}\n//////\n")
        else:
            #Ao contrário somente repetimos a mesma solucão enquanto não acharmos uma melhor
            print(f"Geracão {generation}: Melhor solucão = {melhor_solucao}")

        #No fim atualizamos nossa lista das melhores solucões e adicionamos nosso último melhor fitness e última melhor solucão.
        melhor_fitness_values.append(melhor_fitness)
        melhores_solucoes.append(melhor_solucao)

        #Iniciamos uma populacão do zero mantendo somente o melhor indivíduo da última
        new_population = [population[0]]

        while len(new_population) < POPULATION_SIZE:

            # Selecão
            parent1, parent2 = random.choices(population[:10], k=2)  # Seleciona aleatoriamente dois indivíduos anteriores do top 10.

            # Cruzamento
            child1 = crossover(parent1, parent2) #Realiza o cruzamento desses indivíduos

            # Mutacão
            child1 = mutate(child1, MUTATION_PROBABILITY,min_cpu_capacity,max_cpu_capacity,min_memory_capacity,max_memory_capacity) #Testa a probabilidade de mutacão e coleta o resultado final.

            #Adiciona na nova populacão a nova crianca
            new_population.append(child1)

        #Seta a nova populacão após finalizar o loop para iniciar tudo novamente.
        population = new_population




Client Demands [(66, 632), (71, 496), (73, 325), (89, 666), (89, 558), (53, 593), (23, 611), (45, 183), (10, 371), (13, 499), (4, 143), (33, 443), (74, 147), (86, 158), (64, 433), (29, 229), (19, 295), (70, 113), (7, 480), (67, 576), (53, 674), (89, 659), (86, 241), (2, 353), (30, 483), (86, 625), (22, 447), (27, 375), (43, 238), (60, 425), (85, 162), (48, 320), (80, 178), (67, 567), (22, 477), (52, 507), (27, 337), (32, 384), (50, 402), (26, 683), (83, 140), (38, 96), (75, 186), (52, 151), (72, 351), (33, 645), (35, 294), (29, 43), (37, 162), (17, 527), (74, 301), (46, 349), (41, 609), (43, 203), (56, 286), (59, 287), (29, 647), (14, 94), (23, 440), (74, 670), (25, 151), (23, 230), (85, 293), (55, 80), (40, 671), (79, 253), (16, 495), (9, 330), (52, 486), (73, 367), (84, 218), (67, 355), (24, 112), (60, 271), (60, 498), (66, 219), (64, 356), (38, 480), (77, 96), (30, 672), (30, 676), (70, 572), (38, 143), (82, 185), (80, 51), (23, 175), (43, 449), (6, 519), (40, 414), (40, 608), (73, 